In [3]:
import pandas 
import numpy as np
import cx_Oracle

sql =       "select psnid SPO_ID, "
sql = sql + " case when length(PSNCUIP) = 11 then PSNCUIP  "
sql = sql + "  else null "
sql = sql + "    end SPO_CUIT ,"
sql = sql + "   PTDID TIPO_DOC,"
sql = sql + "   PSNNUMERODOCUMENTO NRO_DOC, "
sql = sql + "   PSNDENOMINACION||' '||PSNNOMBRE  SPO_DENOMINACION, "
sql = sql + "  decode (PTPID,2,'F',3,'J')  SPO_TIPO_SUJETO_PASIVO, "
sql = sql + "  NULL NRO_CTA,NULL DOMINIO,NULL INS_ID,NULL NRO_LOTE  "
sql = sql + "  from antprod.personas@antprod p "
sql = sql + "  where exists ( "
sql = sql + "  select 1 from antprod.objetos@antprod o join antprod.vinculos_persona_objeto@antprod "
sql = sql + "  v on o.ojtid=v.ojtid  where o.otpid in (241,5,6) and p.psnid=v.psnid)"
sql = sql + "  and not exists ( select 1 from audantprod.aud_personas b "
sql = sql + "   inner join migracion.corridas c on b.id_corrida = c.id_corrida "
sql = sql + "   where b.psnid= p.psnid and b.id_pc not in (118,119,120,121,122,123,124,125)) "
sql = sql + "   and not exists(select 1 from personas_cuitificadas c where p.psnid=c.psnid) "

con = cx_Oracle.connect('<<username>>/<<password>>@<<host>>/<<database>>')
dataset = pandas.read_sql(sql , con)
con.close()

dataset["NRO_DOC"] = dataset["NRO_DOC"].replace(np.nan, 0)
dataset["NRO_DOC"] = dataset['NRO_DOC'].astype(np.int64)

dataset.to_csv('sujetos_a_cuitificar.csv', encoding='utf-8', index=False, sep='\t')